In [1]:
import glob
import os
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM, Embedding
from configs import *
from fetch_data import *
from features_extraction import *
from data_shuffling_split import *
from data_preprocess import *
from ml_modeling import *
from keras_models import *

In [2]:
strat_train_set = read_csv("train/strat_train_set.csv")
strat_train_set = strat_train_set.iloc[:5000]
strat_train_set.head()

Number of instances in the file are:  449033


,id,dialect,dialect_l_encoded,text
0,1056552188082716800,LY,8,توا دوشه الكلاسيكو شن بيتمها وشن بيسكتهم وشن ب...
1,891734969202114560,SY,15,حسابشخصي في احلي من الشحاطه 😂
2,1110565179257954432,SD,14,حسابشخصي موهبه والله 😂 اوع تحاول تطورها تقوم م...
3,1172817955270340608,LB,7,حسابشخصي حسابشخصي 😂 انا صرلي عشر سنين مش مجدده...
4,293253217821790208,QA,12,احلي شعور تكون باجازه وتقوم من الصبح وتمر ع ال...


In [3]:
x_train_text, x_val_text, y_train, y_val = prepare_data(strat_train_set)

The number of instances in the training data after StratifiedShuffleSplit are:  4900
The number of instances in the testing data after StratifiedShuffleSplit are:   100
The number of trainin instances:  4900
The number of validation instances:  100
The number of trainin labels :  4900
The number of validation labels :  100


In [4]:
x_train_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_train_text)

print("Before Tokenization : \n", x_train_text[:3])
print("="*50)
print("After Tokenization : \n", x_train_text_tokenized[:3])
print("="*50)

x_val_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_val_text)

print("Before Tokenization : \n", x_val_text[:3])
print("="*50)
print("After Tokenization : \n", x_val_text_tokenized[:3])

Before Tokenization : 
 ['حسابشخصي الله يحشرك واللي وراك معاه بنار جهنم تحرقكم سواد وجوهكم اكثر من ما هي سودا . . . شواذي 🐒 ', 'حسابشخصي متفكريناش صافي زعما سالات العطله 😥 ', 'حاشرين نفسكم في اي بطيخ رابطويب']
After Tokenization : 
 [['حسابشخصي', 'الله', 'يحشرك', 'واللي', 'وراك', 'معاه', 'بنار', 'جهنم', 'تحرقكم', 'سواد', 'وجوهكم', 'اكثر', 'من', 'ما', 'هي', 'سودا', '.', '.', '.', 'شواذي', '🐒'], ['حسابشخصي', 'متفكريناش', 'صافي', 'زعما', 'سالات', 'العطله', '😥'], ['حاشرين', 'نفسكم', 'في', 'اي', 'بطيخ', 'رابطويب']]
Before Tokenization : 
 ['الف الف مبرووك ياشبااب 💚 ', 'حسابشخصي حسابشخصي قمر لو كمره هه', 'حسابشخصي حسابشخصي تري اطلع ملفك من محكمه البحرين واخرته من اهلك ام عبير 😂 ماعاد الا جزيره الريتويت انتم اصلا محد يدري عنكم . . ']
After Tokenization : 
 [['الف', 'الف', 'مبرووك', 'ياشبااب', '💚'], ['حسابشخصي', 'حسابشخصي', 'قمر', 'لو', 'كمره', 'هه'], ['حسابشخصي', 'حسابشخصي', 'تري', 'اطلع', 'ملفك', 'من', 'محكمه', 'البحرين', 'واخرته', 'من', 'اهلك', 'ام', 'عبير', '😂', 'ماعاد', 'الا', 'جزيره', 'ا

# LSTM Model with Bakr Word2vec

In [5]:
word_to_vec_model = load_word2vec_model("models/word2vec/bakrianoo_unigram_cbow_model/full_uni_cbow_100_twitter.mdl")

In [6]:
max_len_str = 64
hid_num_neurons = 50
learning_rate = .1
epochs = 30

performance_lr = keras.callbacks.ReduceLROnPlateau(factor=.5, patience=5)
SGD_optimizer     =keras.optimizers.SGD(learning_rate=learning_rate)
Adam_optimizer = keras.optimizers.Adam(beta_1=0.9, beta_2=0.999)
RMSprop_optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate, rho=.9)


callbacks_ = keras_callbacks(word2vec_type="bakr_word2vec", model_type="lstm_no_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)

number_of_features = 100
word2vec_path = "bakr/"

X_train_embed_matrix = text_to_matrix_using_word2vec(word_to_vec_model, x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(word_to_vec_model, x_val_text_tokenized, max_len_str)

# Reshape because of deep learning model
X_train_embed_matrix = X_train_embed_matrix.reshape(X_train_embed_matrix.shape[0], max_len_str, number_of_features)
X_val_embed_matrix = X_val_embed_matrix.reshape(X_val_embed_matrix.shape[0], max_len_str, number_of_features)

(4900, 64, 100)
(4900, 6400)
[ 5.44    -1.274    3.229   -3.773   -3.795    4.926   -0.6597   2.365
  0.5625  -3.352    0.3213   0.5005  -3.709   -5.33     3.615    1.654
  2.988   -0.7603   0.3733   2.418   -2.904   -2.7      2.135   -0.3982
  0.63     1.314   -0.8735   4.855    4.684   -1.277    3.67    -2.932
  2.867    2.352    2.062   -0.756   -2.1      5.863    0.9263  -0.7715
  0.5312   4.836   -0.761   -0.4324  -3.611   -1.574    0.10724 -3.316
 -1.546    3.238  ]
(100, 64, 100)
(100, 6400)
[-5.2197e-01 -3.3154e-01  3.4512e+00  1.1807e+00  4.6753e-01 -1.2539e+00
  8.2861e-01  3.2012e+00 -8.2764e-01  3.2363e+00 -1.9795e+00 -4.4961e+00
 -3.3965e+00 -4.1953e+00 -6.5674e-01  1.0615e+00 -2.7148e+00  7.0352e+00
 -3.2754e+00 -2.6641e+00 -1.9702e-01  1.3691e+00  3.8354e-01  3.5625e+00
  5.5469e-01 -5.5078e-01 -2.3594e+00  3.8330e-01  9.3311e-01 -5.9912e-01
 -2.7500e+00  7.1240e-01  1.2734e+00 -4.3848e-01 -4.6997e-01 -7.8516e-01
 -1.2666e+00  2.4297e+00 -1.8193e+00  1.1855e+00 -3.5996e+

# With  SGD and No Batch Normalization

In [7]:
callbacks_ = keras_callbacks(word2vec_type="bakr_word2vec", model_type="sgd_lstm_no_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_no_batch_seqential_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, SGD_optimizer)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64, 50)            30200     
_________________________________________________________________
dropout (Dropout)            (None, 64, 50)            0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 18)                57618     
Total params: 87,818
Trainable params: 87,818
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/30
154/154 [==============================] - 6s 28ms/step - loss: 2.5555 - accuracy: 0.2055 - val_loss: 2.2764 - val_accuracy: 0.2600
Epoch 2/30
154/154 [==============================] - 4s 25ms/step - loss: 2.1528 - accuracy: 0.3241 - val_loss: 2.3268 - val_accuracy: 0.2400
Epoch 3/30
154/154 [==============================] - 4s 25ms/step - loss: 1.9022 - accuracy: 0.3953 - val_loss: 2.3028 - val_accuracy: 0.2700
Epoch 4/30
154/154 [==============================] - 4s 25ms/step - loss: 1.6945 - accuracy: 0.4651 - val_loss: 2.2819 - val_accuracy: 0.3000
Epoch 5/30
154/154 [==============================] - 4s 25ms/step - loss: 1.5218 - accuracy: 0.5304 - val_loss: 2.3182 - val_accuracy: 0.3100
Epoch 6/30
154/154 [==============================] - 4s 25ms/step - loss: 1.3570 - accuracy: 0.5884 - val_loss: 2.4854 - val_accuracy: 0.2800


# With  Adam and No Batch Normalization

In [9]:
callbacks_ = keras_callbacks(word2vec_type="bakr_word2vec", model_type="Adam_lstm_no_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_no_batch_seqential_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, Adam_optimizer)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64, 50)            30200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 50)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 18)                57618     
Total params: 87,818
Trainable params: 87,818
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/30
154/154 [==============================] - 6s 29ms/step - loss: 2.5348 - accuracy: 0.2039 - val_loss: 2.2425 - val_accuracy: 0.2700
Epoch 2/30
154/154 [==============================] - 4s 25ms/step - loss: 2.0438 - accuracy: 0.3549 - val_loss: 2.2127 - val_accuracy: 0.3100
Epoch 3/30
154/154 [==============================] - 4s 25ms/step - loss: 1.7476 - accuracy: 0.4480 - val_loss: 2.1682 - val_accuracy: 0.3200
Epoch 4/30
154/154 [==============================] - 4s 26ms/step - loss: 1.5219 - accuracy: 0.5278 - val_loss: 2.1900 - val_accuracy: 0.3500
Epoch 5/30
154/154 [==============================] - 4s 26ms/step - loss: 1.3329 - accuracy: 0.5880 - val_loss: 2.2618 - val_accuracy: 0.3400
Epoch 6/30
154/154 [==============================] - 4s 25ms/step - loss: 1.1610 - accuracy: 0.6455 - val_loss: 2.3278 - val_accuracy: 0.3400
Epoch 7/30
154/154 [==============================] - 4s 26ms/step - loss: 1.0069 - accuracy: 0.7018 - val_loss: 2.5368 - val_accuracy: 0.3200

# With  RMSprob and No Batch Normalization

In [11]:
callbacks_ = keras_callbacks(word2vec_type="bakr_word2vec", model_type="Rmsprob_lstm_no_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_no_batch_seqential_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, RMSprop_optimizer)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 64, 50)            30200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 50)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 18)                57618     
Total params: 87,818
Trainable params: 87,818
Non-trainable params: 0
_________________________________________________________________


In [12]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/30
154/154 [==============================] - 6s 29ms/step - loss: 28.1005 - accuracy: 0.1014 - val_loss: 6.7488 - val_accuracy: 0.1400
Epoch 2/30
154/154 [==============================] - 4s 25ms/step - loss: 4.9913 - accuracy: 0.1545 - val_loss: 4.1534 - val_accuracy: 0.1600
Epoch 3/30
154/154 [==============================] - 4s 26ms/step - loss: 3.7764 - accuracy: 0.1867 - val_loss: 3.9269 - val_accuracy: 0.1700
Epoch 4/30
154/154 [==============================] - 4s 26ms/step - loss: 3.4448 - accuracy: 0.2045 - val_loss: 3.9435 - val_accuracy: 0.2300
Epoch 5/30
154/154 [==============================] - 4s 26ms/step - loss: 3.2215 - accuracy: 0.2312 - val_loss: 4.1335 - val_accuracy: 0.1900
Epoch 6/30
154/154 [==============================] - 4s 26ms/step - loss: 3.0990 - accuracy: 0.2433 - val_loss: 3.9438 - val_accuracy: 0.1800
Epoch 7/30
154/154 [==============================] - 4s 27ms/step - loss: 2.9353 - accuracy: 0.2690 - val_loss: 4.1138 - val_accuracy: 0.190

# With  SGD and Batch Normalization

In [14]:
callbacks_ = keras_callbacks(word2vec_type="bakr_word2vec", model_type="sgd_lstm_with_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, SGD_optimizer)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 64, 100)           400       
_________________________________________________________________
lstm_3 (LSTM)                (None, 64, 50)            30200     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 50)            200       
_________________________________________________________________
dropout_3 (Dropout)          (None, 64, 50)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 18)                57618     
Total params: 88,418
Trainable params: 88,118
Non-trainable params: 300
________________________________________________

In [15]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/30
154/154 [==============================] - 7s 34ms/step - loss: 3.3547 - accuracy: 0.1867 - val_loss: 2.5118 - val_accuracy: 0.2100
Epoch 2/30
154/154 [==============================] - 5s 30ms/step - loss: 1.9984 - accuracy: 0.3949 - val_loss: 2.4229 - val_accuracy: 0.2800
Epoch 3/30
154/154 [==============================] - 5s 30ms/step - loss: 1.5653 - accuracy: 0.5078 - val_loss: 2.5726 - val_accuracy: 0.2500
Epoch 4/30
154/154 [==============================] - 5s 30ms/step - loss: 1.3079 - accuracy: 0.5753 - val_loss: 2.8895 - val_accuracy: 0.2600
Epoch 5/30
154/154 [==============================] - 5s 30ms/step - loss: 1.1259 - accuracy: 0.6388 - val_loss: 2.8880 - val_accuracy: 0.2800
Epoch 6/30
154/154 [==============================] - 5s 30ms/step - loss: 0.9814 - accuracy: 0.6888 - val_loss: 3.3794 - val_accuracy: 0.2500
Epoch 7/30
154/154 [==============================] - 5s 30ms/step - loss: 0.8909 - accuracy: 0.7194 - val_loss: 3.3187 - val_accuracy: 0.2900

# With  Adam and No Batch Normalization

In [16]:
callbacks_ = keras_callbacks(word2vec_type="bakr_word2vec", model_type="Adam_lstm_with_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, Adam_optimizer)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 64, 100)           400       
_________________________________________________________________
lstm_4 (LSTM)                (None, 64, 50)            30200     
_________________________________________________________________
batch_normalization_3 (Batch (None, 64, 50)            200       
_________________________________________________________________
dropout_4 (Dropout)          (None, 64, 50)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 18)                57618     
Total params: 88,418
Trainable params: 88,118
Non-trainable params: 300
________________________________________________

In [17]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/30
154/154 [==============================] - 7s 34ms/step - loss: 3.1715 - accuracy: 0.1882 - val_loss: 2.3764 - val_accuracy: 0.3300
Epoch 2/30
154/154 [==============================] - 5s 30ms/step - loss: 1.8332 - accuracy: 0.4218 - val_loss: 2.3358 - val_accuracy: 0.3000
Epoch 3/30
154/154 [==============================] - 5s 30ms/step - loss: 1.4818 - accuracy: 0.5331 - val_loss: 2.4593 - val_accuracy: 0.3000
Epoch 4/30
154/154 [==============================] - 5s 33ms/step - loss: 1.2800 - accuracy: 0.5931 - val_loss: 2.5501 - val_accuracy: 0.3200
Epoch 5/30
154/154 [==============================] - 5s 30ms/step - loss: 1.1208 - accuracy: 0.6463 - val_loss: 2.6472 - val_accuracy: 0.3100
Epoch 6/30
154/154 [==============================] - 5s 30ms/step - loss: 0.9951 - accuracy: 0.6959 - val_loss: 2.8590 - val_accuracy: 0.3100
Epoch 7/30
154/154 [==============================] - 5s 30ms/step - loss: 0.8874 - accuracy: 0.7257 - val_loss: 2.8334 - val_accuracy: 0.3300

# With  RMSprob and No Batch Normalization

In [18]:
callbacks_ = keras_callbacks(word2vec_type="bakr_word2vec", model_type="Rmsprob_lstm_with_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, RMSprop_optimizer)
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 64, 100)           400       
_________________________________________________________________
lstm_5 (LSTM)                (None, 64, 50)            30200     
_________________________________________________________________
batch_normalization_5 (Batch (None, 64, 50)            200       
_________________________________________________________________
dropout_5 (Dropout)          (None, 64, 50)            0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 18)                57618     
Total params: 88,418
Trainable params: 88,118
Non-trainable params: 300
________________________________________________

In [19]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/30
154/154 [==============================] - 7s 34ms/step - loss: 13.1241 - accuracy: 0.0931 - val_loss: 20.3543 - val_accuracy: 0.0200
Epoch 2/30
154/154 [==============================] - 5s 30ms/step - loss: 9.0058 - accuracy: 0.0986 - val_loss: 21.7042 - val_accuracy: 0.0500
Epoch 3/30
154/154 [==============================] - 5s 30ms/step - loss: 8.8351 - accuracy: 0.1061 - val_loss: 12.7319 - val_accuracy: 0.0400
Epoch 4/30
154/154 [==============================] - 5s 31ms/step - loss: 8.9997 - accuracy: 0.1010 - val_loss: 20.5070 - val_accuracy: 0.0600
Epoch 5/30
154/154 [==============================] - 5s 30ms/step - loss: 9.4414 - accuracy: 0.1033 - val_loss: 10.4307 - val_accuracy: 0.1300
Epoch 6/30
154/154 [==============================] - 5s 30ms/step - loss: 10.4542 - accuracy: 0.0924 - val_loss: 9.6145 - val_accuracy: 0.0200
Epoch 7/30
154/154 [==============================] - 5s 30ms/step - loss: 9.4937 - accuracy: 0.0924 - val_loss: 12.4146 - val_accuracy

# LSTM Model with Rezk Word2vec

In [20]:
word_to_vec_model = load_word2vec_model("models/word2vec/rezk_unigram_CBOW_model/train_word2vec_cbow__window_3_min_count_300")

In [21]:
number_of_features = 300

callbacks_ = keras_callbacks(word2vec_type="rezk_word2vec", model_type="lstm_no_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)

word2vec_path = "rezk/"

X_train_embed_matrix = text_to_matrix_using_word2vec(word_to_vec_model, x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(word_to_vec_model, x_val_text_tokenized, max_len_str)

# Reshape because of deep learning model
X_train_embed_matrix = X_train_embed_matrix.reshape(X_train_embed_matrix.shape[0], max_len_str, number_of_features)
X_val_embed_matrix = X_val_embed_matrix.reshape(X_val_embed_matrix.shape[0], max_len_str, number_of_features)

(4900, 64, 300)
(4900, 19200)
[-0.1262   0.2761   0.2466  -0.3464  -0.5044   0.216    0.2651   0.05423
 -0.3276  -0.2793   0.328    0.1699  -0.05267  0.1941   0.292    0.1654
 -0.01619 -0.428    0.411    0.0927   0.271    0.6206  -0.04764  0.04465
  0.0863   0.06042  0.08374 -0.0927   0.05176 -0.1616  -0.4875   0.4932
  0.1333   0.4666   0.0387  -0.19     0.05563 -0.1526   0.549    0.2966
 -0.0969  -0.345   -0.2896  -0.0667   0.12146  0.2126   0.1146  -0.4404
 -0.1198   0.2651 ]
(100, 64, 300)
(100, 19200)
[-1.091    0.2328   0.1113  -0.4392   0.687   -0.8223  -0.572   -0.454
  0.2      0.3171  -0.5127  -0.2296   0.2198  -0.07245 -0.548    0.3447
  0.2394   0.2744   0.497   -1.322   -0.6577   0.294    0.965    0.6753
  0.6943   0.0327  -1.08    -0.3694  -0.4783   0.351    0.7593   0.07
 -0.284   -0.422   -0.639    0.405    0.7314  -0.171   -0.0849   0.1683
  1.39    -1.2     -0.631   -0.5005   0.748   -0.698    0.542   -0.6484
 -0.12427 -0.607  ]


# With  SGD and No Batch Normalization

In [22]:
callbacks_ = keras_callbacks(word2vec_type="rezk_word2vec", model_type="sgd_lstm_no_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_no_batch_seqential_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, SGD_optimizer)
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 64, 50)            70200     
_________________________________________________________________
dropout_6 (Dropout)          (None, 64, 50)            0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 18)                57618     
Total params: 127,818
Trainable params: 127,818
Non-trainable params: 0
_________________________________________________________________


In [23]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/30
154/154 [==============================] - 7s 37ms/step - loss: 2.7574 - accuracy: 0.1484 - val_loss: 2.6179 - val_accuracy: 0.2500
Epoch 2/30
154/154 [==============================] - 5s 33ms/step - loss: 2.5406 - accuracy: 0.2294 - val_loss: 2.4228 - val_accuracy: 0.2400
Epoch 3/30
154/154 [==============================] - 5s 34ms/step - loss: 2.3578 - accuracy: 0.2694 - val_loss: 2.2620 - val_accuracy: 0.2600
Epoch 4/30
154/154 [==============================] - 5s 35ms/step - loss: 2.2158 - accuracy: 0.3037 - val_loss: 2.1814 - val_accuracy: 0.2900
Epoch 5/30
154/154 [==============================] - 5s 34ms/step - loss: 2.1019 - accuracy: 0.3292 - val_loss: 2.1751 - val_accuracy: 0.2600
Epoch 6/30
154/154 [==============================] - 5s 31ms/step - loss: 2.0112 - accuracy: 0.3496 - val_loss: 2.3050 - val_accuracy: 0.2600
Epoch 7/30
154/154 [==============================] - 5s 31ms/step - loss: 1.9406 - accuracy: 0.3802 - val_loss: 2.0963 - val_accuracy: 0.3300

# With  Adam and No Batch Normalization

In [24]:
callbacks_ = keras_callbacks(word2vec_type="rezk_word2vec", model_type="Adam_lstm_no_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_no_batch_seqential_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, Adam_optimizer)
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 64, 50)            70200     
_________________________________________________________________
dropout_7 (Dropout)          (None, 64, 50)            0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 18)                57618     
Total params: 127,818
Trainable params: 127,818
Non-trainable params: 0
_________________________________________________________________


In [25]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/30
154/154 [==============================] - 9s 43ms/step - loss: 2.3610 - accuracy: 0.2592 - val_loss: 2.0594 - val_accuracy: 0.3000
Epoch 2/30
154/154 [==============================] - 8s 52ms/step - loss: 1.8890 - accuracy: 0.3982 - val_loss: 1.9170 - val_accuracy: 0.3600
Epoch 3/30
154/154 [==============================] - 9s 60ms/step - loss: 1.6939 - accuracy: 0.4563 - val_loss: 1.8842 - val_accuracy: 0.3500
Epoch 4/30
154/154 [==============================] - 7s 46ms/step - loss: 1.5359 - accuracy: 0.5112 - val_loss: 1.8664 - val_accuracy: 0.3900
Epoch 5/30
154/154 [==============================] - 5s 35ms/step - loss: 1.4031 - accuracy: 0.5516 - val_loss: 1.8713 - val_accuracy: 0.3700
Epoch 6/30
154/154 [==============================] - 6s 37ms/step - loss: 1.2765 - accuracy: 0.5990 - val_loss: 1.9142 - val_accuracy: 0.4200
Epoch 7/30
154/154 [==============================] - 5s 35ms/step - loss: 1.1719 - accuracy: 0.6302 - val_loss: 1.9494 - val_accuracy: 0.3400

# With  Rmsprob and No Batch Normalization

In [26]:
callbacks_ = keras_callbacks(word2vec_type="rezk_word2vec", model_type="Rmsprob_lstm_no_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_no_batch_seqential_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, RMSprop_optimizer)
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 64, 50)            70200     
_________________________________________________________________
dropout_8 (Dropout)          (None, 64, 50)            0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 18)                57618     
Total params: 127,818
Trainable params: 127,818
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/30
154/154 [==============================] - 7s 35ms/step - loss: 3.7504 - accuracy: 0.1912 - val_loss: 2.7484 - val_accuracy: 0.2400
Epoch 2/30
154/154 [==============================] - 6s 37ms/step - loss: 2.1191 - accuracy: 0.3633 - val_loss: 2.7736 - val_accuracy: 0.2300
Epoch 3/30
154/154 [==============================] - 5s 31ms/step - loss: 1.6886 - accuracy: 0.4867 - val_loss: 2.9992 - val_accuracy: 0.2600
Epoch 4/30
154/154 [==============================] - 5s 31ms/step - loss: 1.3692 - accuracy: 0.5747 - val_loss: 3.3729 - val_accuracy: 0.2800
Epoch 5/30
119/154 [======================>.......] - ETA: 1s - loss: 1.0784 - accuracy: 0.6549

# With  SGD and  Batch Normalization

In [ ]:
callbacks_ = keras_callbacks(word2vec_type="rezk_word2vec", model_type="sgd_lstm_with_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, SGD_optimizer)
model.summary()

In [ ]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

# With  Adam and  Batch Normalization

In [ ]:
callbacks_ = keras_callbacks(word2vec_type="rezk_word2vec", model_type="Adam_lstm_with_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, Adam_optimizer)
model.summary()

In [ ]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

# With  Rmsprob and  Batch Normalization

In [ ]:
callbacks_ = keras_callbacks(word2vec_type="rezk_word2vec", model_type="Rmsprob_lstm_with_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, RMSprop_optimizer)
model.summary()

In [ ]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)